In [1]:
__nbid__ = '0001'
__author__ = 'Sylvie Dagoret-Campagne <sylvie.dagoret-campagne@ijclab.in2p3.fr>'
__version__ = '20241230' # yyyymmdd
__datasets__ = ['desi_edr']  
__keywords__ = ['sparcl', 'spectra', 'database']

- Last update : 2024-12-30

# Access to DESI BGS Spectra for PZ SED Templates

Inspired from astro datalab Noirlab notebooks

<a class="anchor" id="import"></a>
# Imports

Note in this version we avoid special **desitarget** and **desiutil** tools
We only use **sparcl** to retrieve spectra

In [2]:
## import some helpful python packages 
import numpy as np
import pandas as pd

from astropy.table import Table
from astropy.convolution import convolve, Gaussian1DKernel

import matplotlib 
import matplotlib.pyplot as plt

%matplotlib inline

## Data Lab related modules
from sparcl.client import SparclClient
from dl import queryClient as qc, authClient as ac
from getpass import getpass

In [3]:
import astropy.units as u

In [4]:
#F0AB = 3631*(1*u.Jy).cgs.value
F0AB = 3631*(1*u.Jy)

<a class="anchor" id="auth"></a>
# Authentication
If you need to log in to Data Lab, un-comment the cell below and execute it:

In [5]:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

'sdagoret'

In [6]:
## Making the matplotlib plots look nicer
settings = {
    'font.size':22,
    'axes.linewidth':3.0,
    'xtick.major.size':6.0,
    'xtick.minor.size':4.0,
    'xtick.major.width':3.0,
    'xtick.minor.width':1.5,
    'xtick.direction':'in', 
    'xtick.minor.visible':True,
    'xtick.top':True,
    'ytick.major.size':6.0,
    'ytick.minor.size':4.0,
    'ytick.major.width':3.0,
    'ytick.minor.width':1.5,
    'ytick.direction':'in', 
    'ytick.minor.visible':True,
    'ytick.right':True
}

plt.rcParams.update(**settings)

In [7]:
## Instantiate SPARCL Client
client = SparclClient()

<a class="anchor" id="desi_edr"></a>
# DESI EDR database

The `desi_edr` database schema at Data Lab consists of several tables associated with the targets.

In [8]:
print(qc.schema('desi_edr'))


Schema: desi_edr

      Table Name   Description
      ----------   -----------
        exposure   Summary quantities for every DESI exposure
     fiberassign   Quantities obtained when a DESI target is assigned to a fiber
           frame   Summary quantities for each petal of the DESI instrument i
                   n a given exposure; in normal operation there are ten fram
                   es for every exposure
      photometry   Photometric quantities from LS DR9 for every TARGETID
       potential   For a given tile, this table lists all targets that could 
                   have received a fiber assignment
          target   The quantities obtained when photometric objects are analy
                   zed in the target selection process
            tile   Summary quantities for every DESI tile (pointing on the sky)
x1p5__zpix__allwise__source   desi_edr.zpix and xmatch_std.allwise__source crossmatch (1
                   .5 arcsec)
x1p5__zpix__gaia_dr3__gaia_source   desi_edr

We focus on the `photometry` and `zpix` table in this notebook, which is the main redshift catalog from the DESI survey. It contains the redshift information from the coadded-spectra, based on the healpix of the objects.

We look at the available columns in this table.

### Photometry table

In [9]:
## Columns in the photometry table
print(qc.schema('desi_edr.photometry'))


Schema: desi_edr
 Table: photometry

     Column Name   Description
     -----------   -----------
              ra   Right ascension
             dec   Declination
            elon   Ecliptic longitude
            elat   Ecliptic latitude
            glon   Galactic longitude
            glat   Galactic latitude
           ls_id   Unique Legacy Survey object ID
          ref_id   Tyc1*1000000+Tyc2*10+Tyc3 for Tycho-2; sourceid for Gaia DR2
        targetid   Unique DESI target ID
         ra_ivar   Right ascension inverse variance
        dec_ivar   Declination inverse variance
      dchisq_psf   Difference in chi-squared between PSF (stellar) Tractor mo
                   del fits
      dchisq_rex   Difference in chi-squared between REX (round exponential g
                   alaxy) Tractor model fits
      dchisq_dev   Difference in chi-squared between DEV (deVauc) Tractor mod
                   el fits
      dchisq_exp   Difference in chi-squared between EXP (exponential) Tracto
 

### zpix table

In [10]:
## Columns in the zpix table
print(qc.schema('desi_edr.zpix'))


Schema: desi_edr
 Table: zpix

     Column Name   Description
     -----------   -----------
 tsnr2_gpbdark_r   GPBDARK R template (S/N)^2
     tsnr2_elg_r   ELG R template (S/N)^2
tsnr2_gpbbright_r   GPBBRIGHT R template (S/N)^2
     tsnr2_lya_r   LYA R template (S/N)^2
               z   Redshift measured by Redrock
            zerr   Redshift error from Redrock
            chi2   Best fit chi squared
         coeff_0   Redrock template coefficients
         coeff_1   Redrock template coefficients
         coeff_2   Redrock template coefficients
         coeff_3   Redrock template coefficients
         coeff_4   Redrock template coefficients
         coeff_5   Redrock template coefficients
         coeff_6   Redrock template coefficients
         coeff_7   Redrock template coefficients
         coeff_8   Redrock template coefficients
         coeff_9   Redrock template coefficients
       deltachi2   Delta-chi-squared for template fit from Redrock
   mean_fiber_ra   Mean (over expos

<a class="anchor" id="desi_target_access"></a>
# Accessing DESI targets

The DESI spectra are uniquely identified by three quantities:

* **targetid**: unique identifier for a given target
* **survey**: SURVEY that the target was observed in. This can be _cmx_, _special_, _sv1_, _sv2_, or _sv3_.
* **program**: FIBER ASSIGNMENT PROGRAM. This is the planned observing conditions for the target. It can be _dark_ or _bright_ or _backup_. In case of _cmx_ and _sv1_, there is _other_ as well.
    
The different spectra of individual targets are coadded within each survey and program. Therefore, some targets may have multiple coadded spectra and each one is associated with a given `targetid`, `survey`, and `program`. The "best" spectrum for a given object is given by the `zcat_primary` column. More information about the data model of the redshift catalogs is available [here](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zpix-SURVEY-PROGRAM.html#).

In [11]:
#eg. SELECT (column_1, column_2, ... column_n) FROM T1 JOIN T2 ON (condition) WHERE (condition)

### To select in two tables

```
SELECT t1.id, t1.ra, t1.dec, t2.mag, t2.band
FROM catalog.table1 AS t1
JOIN catalog.table2 AS t2 ON t1.id = t2.id
WHERE CONTAINS(POINT('IRCS', t1.ra, t1.dec), CIRCLE('ICRS', 62.123, -37.456 0.002)) = 1
```

In [12]:
query = """
SELECT zp.targetid, zp.survey, zp.program, zp.healpix,  
       zp.z, zp.zwarn, zp.coadd_fiberstatus, zp.spectype, 
       zp.mean_fiber_ra, zp.mean_fiber_dec, zp.zcat_nspec, 
       CAST(zp.zcat_primary as int), zp.desi_target,
       zp.sv1_desi_target, zp.sv2_desi_target, zp.sv3_desi_target
FROM desi_edr.zpix AS zp
""" 

## Information about the query
## Selected columns --
## targetid, survey, program -- unique identifiers for a given spectrum
## healpix -- healpix number for the target
## z -- spectroscopic redshift of the target
## zwarn -- encoded information regarding the redshift (zwarn = 0 is good)
## coadd_fiberstatus -- encoded information regarding the fiber that is assigned to the target (coadd_fiberstatus = 0 is good)
## spectype -- Spectral type of the target: STAR | GALAXY | QSO
## mean_fiber_ra, mean_fiber_dec -- Mean R.A. and Dec. of the fiber position from all the observations of the target
## zcat_nspec -- Number of coadded spectra that are available for a given target
## zcat_primary -- Whether or not a given coadded spectrum is the primary spectrum. zcat_primary = True for the "best" spectrum.
## CASTing this column as an INT: zcat_primary = 1 for the "best" spectrum.
## desi_target -- encodes main survey's DESI targeting information - explained in detail below
## sv1_desi_target -- encodes sv1 desi targeting information
## sv2_desi_target -- encodes sv2 desi targeting information
## sv3_desi_target -- encodes sv3 desi targeting information

In [13]:
query = """
SELECT zp.targetid, zp.survey, zp.program, zp.healpix,  
       zp.z, zp.zwarn, zp.coadd_fiberstatus, zp.spectype, 
       zp.mean_fiber_ra, zp.mean_fiber_dec, zp.zcat_nspec, 
       CAST(zp.zcat_primary as int), zp.desi_target,
       zp.sv1_desi_target, zp.sv2_desi_target, zp.sv3_desi_target,
       ph.ra,ph.dec, ph.morphtype,
       ph.flux_g,ph.flux_r, ph.flux_z, ph.flux_ivar_g, ph.flux_ivar_r,ph.flux_ivar_z,
       ph.flux_w1, ph.flux_w2, ph.flux_w3, ph.flux_w4, 
       ph.flux_ivar_w1, ph.flux_ivar_w2, ph.flux_ivar_w3, ph.flux_ivar_w4   
FROM desi_edr.zpix AS zp JOIN desi_edr.photometry AS ph ON (zp.targetid = ph.targetid)
""" 

In [14]:
print(query)


SELECT zp.targetid, zp.survey, zp.program, zp.healpix,  
       zp.z, zp.zwarn, zp.coadd_fiberstatus, zp.spectype, 
       zp.mean_fiber_ra, zp.mean_fiber_dec, zp.zcat_nspec, 
       CAST(zp.zcat_primary as int), zp.desi_target,
       zp.sv1_desi_target, zp.sv2_desi_target, zp.sv3_desi_target,
       ph.ra,ph.dec, ph.morphtype,
       ph.flux_g,ph.flux_r, ph.flux_z, ph.flux_ivar_g, ph.flux_ivar_r,ph.flux_ivar_z,
       ph.flux_w1, ph.flux_w2, ph.flux_w3, ph.flux_w4, 
       ph.flux_ivar_w1, ph.flux_ivar_w2, ph.flux_ivar_w3, ph.flux_ivar_w4   
FROM desi_edr.zpix AS zp JOIN desi_edr.photometry AS ph ON (zp.targetid = ph.targetid)



In [ ]:
zpix = qc.query(sql = query, fmt = 'table')

In [ ]:
print(f"The resulting table has {len(zpix)} rows")
zpix[0:5]

In [ ]:
# Check how many rows have unique TARGETIDs before/after applying the ZCAT_PRIMARY flag
print(f"Total N(rows) : {len(zpix)}")
print(f"N(rows) with unique TARGETIDs : {len(np.unique(zpix['targetid']))}")

is_primary = (zpix['zcat_primary']==1)

print(f"N(rows) with ZCAT_PRIMARY=True : {len(zpix[is_primary])}")

In [ ]:
## Selecting only unique objects
zpix_cat = zpix[is_primary]

In [ ]:
type(zpix_cat)

In [ ]:
df = zpix_cat.to_pandas()

In [ ]:
df.describe()

### Remove Galaxies with unknown mags and select Galaxies only

In [ ]:
cut = (df.flux_g == 0) | (df.flux_r == 0) | (df.flux_z == 0) |  (df.flux_w1 == 0) | (df.flux_w2 == 0)
df = df.drop(df[cut].index)

In [ ]:
df = df[df.spectype == "GALAXY"]

### Convert fluxes in AB magnitudes
- reference on the Legacy Survey here : https://www.legacysurvey.org/dr8/description/

- a flux of 1 nanomaggie corresponds to an AB magnitude of 22.5.
- A source with a spectrum of 𝑓 = 10^( − (48.6 + 22.5)/2.5) erg/s/cm²/Hz would be reported to have an integrated flux of 1 nanomaggie in any filter
- a flux of one maggies is constant in erg/s/cm²/Hz

In [ ]:
#dm = -2.5 / log(10) * 1/(sqrt(ivar)*f)
error_factor = 2.5/np.log(10)

In [ ]:
# assuming the flux in in Jy
#df["mag_g"] = df["flux_g"].apply(lambda x : (x*u.Jy).to_value(u.ABmag))
#df["mag_r"] = df["flux_r"].apply(lambda x : (x*u.Jy).to_value(u.ABmag))
#df["mag_z"] = df["flux_z"].apply(lambda x : (x*u.Jy).to_value(u.ABmag))

# assuming the flux is in nanomaggies
#df["mag_g"] = df["flux_g"].apply(lambda x : 22.5 - 2.5*np.log10(x))
#df["mag_r"] = df["flux_r"].apply(lambda x : 22.5 - 2.5*np.log10(x))
#df["mag_z"] = df["flux_z"].apply(lambda x : 22.5 - 2.5*np.log10(x))

# assuming the flux is in maggies (erg/s/cm²/Hz)
df["mag_g"] = df["flux_g"].apply(lambda x : (x*u.erg/u.s/(u.cm)**2/u.Hz).to_value(u.ABmag) + 48.6 + 22.5)
df["mag_r"] = df["flux_r"].apply(lambda x : (x*u.erg/u.s/(u.cm)**2/u.Hz).to_value(u.ABmag) + 48.6 + 22.5)
df["mag_z"] = df["flux_z"].apply(lambda x : (x*u.erg/u.s/(u.cm)**2/u.Hz).to_value(u.ABmag) + 48.6 + 22.5)

In [ ]:
# assuming the flux is in maggies (erg/s/cm²/Hz)
df["mag_w1"] = df["flux_w1"].apply(lambda x : (x*u.erg/u.s/(u.cm)**2/u.Hz).to_value(u.ABmag) + 48.6 + 22.5)
df["mag_w2"] = df["flux_w2"].apply(lambda x : (x*u.erg/u.s/(u.cm)**2/u.Hz).to_value(u.ABmag) + 48.6 + 22.5)
df["mag_w3"] = df["flux_w3"].apply(lambda x : (x*u.erg/u.s/(u.cm)**2/u.Hz).to_value(u.ABmag) + 48.6 + 22.5)
df["mag_w4"] = df["flux_w4"].apply(lambda x : (x*u.erg/u.s/(u.cm)**2/u.Hz).to_value(u.ABmag) + 48.6 + 22.5)

In [ ]:
df["mag_g_err"] = df[["flux_g","flux_ivar_g"]].apply(lambda x : error_factor/x[0]/np.sqrt(x[1]),raw=True,axis=1)
df["mag_r_err"] = df[["flux_r","flux_ivar_r"]].apply(lambda x : error_factor/x[0]/np.sqrt(x[1]),raw=True,axis=1)
df["mag_z_err"] = df[["flux_z","flux_ivar_z"]].apply(lambda x : error_factor/x[0]/np.sqrt(x[1]),raw=True,axis=1)

In [ ]:
df["mag_w1_err"] = df[["flux_w1","flux_ivar_w1"]].apply(lambda x : error_factor/x[0]/np.sqrt(x[1]),raw=True,axis=1)
df["mag_w2_err"] = df[["flux_w2","flux_ivar_w2"]].apply(lambda x : error_factor/x[0]/np.sqrt(x[1]),raw=True,axis=1)
df["mag_w3_err"] = df[["flux_w3","flux_ivar_w3"]].apply(lambda x : error_factor/x[0]/np.sqrt(x[1]),raw=True,axis=1)
df["mag_w4_err"] = df[["flux_w4","flux_ivar_w4"]].apply(lambda x : error_factor/x[0]/np.sqrt(x[1]),raw=True,axis=1)

### Plot magnitudes

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(16,4),layout="constrained")
ax1,ax2,ax3 = axs.flatten()
df["mag_g"].hist(bins=50,range=(0,32),ax=ax1,facecolor="g")
ax1.set_yscale("log")
df["mag_r"].hist(bins=50,range=(0,32),ax=ax2,facecolor="r")
ax2.set_yscale("log")
df["mag_z"].hist(bins=50,range=(0,32),ax=ax3,facecolor="grey")
ax3.set_yscale("log")

In [ ]:
fig,axs = plt.subplots(1,4,figsize=(18,4),layout="constrained")
ax1,ax2,ax3,ax4 = axs.flatten()
df["mag_w1"].hist(bins=50,range=(0,32),ax=ax1,facecolor="C1",label="w1")
ax1.set_yscale("log")
df["mag_w2"].hist(bins=50,range=(0,32),ax=ax2,facecolor="C2",label="w2")
ax2.set_yscale("log")
df["mag_w3"].hist(bins=50,range=(0,32),ax=ax3,facecolor="C3",label="w3")
ax3.set_yscale("log")
df["mag_w4"].hist(bins=50,range=(0,32),ax=ax4,facecolor="C4",label="w4")
ax4.set_yscale("log")

### Plot magnitudes errors

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(16,4),layout="constrained")
ax1,ax2,ax3 = axs.flatten()
df.plot.scatter(x="mag_g",y="mag_g_err",ax=ax1,marker=".",color="g")
df.plot.scatter(x="mag_r",y="mag_r_err",ax=ax2,marker=".",color="r")
df.plot.scatter(x="mag_z",y="mag_z_err",ax=ax3,marker=".",color="grey")

ax1.set_ylim(0,0.1)
ax2.set_ylim(0,0.1)
ax3.set_ylim(0,0.1)

In [ ]:
fig,axs = plt.subplots(1,4,figsize=(18,4),layout="constrained")
ax1,ax2,ax3,ax4 = axs.flatten()
df.plot.scatter(x="mag_w1",y="mag_w1_err",ax=ax1,marker=".",color="C1")
df.plot.scatter(x="mag_w2",y="mag_w2_err",ax=ax2,marker=".",color="C2")
df.plot.scatter(x="mag_w3",y="mag_w3_err",ax=ax3,marker=".",color="C3")
df.plot.scatter(x="mag_w4",y="mag_w4_err",ax=ax4,marker=".",color="C4")

ax1.set_ylim(0,0.1)
ax2.set_ylim(0,0.1)
ax3.set_ylim(0,0.1)
ax4.set_ylim(0,0.1)

## Localisation in Sky

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,6),layout="constrained")
df.plot.scatter(x="ra",y="dec",ax=ax,marker=".",color="b",alpha=0.8,grid=True)
ax.legend(bbox_to_anchor=(1.1, 1.05))
plt.gca().set_aspect('equal')

<a class="anchor" id="target_info"></a>
## Selecting sources based on targeting information

DESI targets are divided into five main classes:
* MWS: Milky Way Survey - This survey probes "stars" in the Milky Way.
* BGS: Bright Galaxy Survey - This survey probes the bright galaxies in the nearby universe.
* LRG: Luminous Red Galaxies
* ELG: Emission Line Galaxies
* QSO: Quasars

[Myers et al. 2023](https://ui.adsabs.harvard.edu/abs/2023AJ....165...50M/abstract) describe the target selection and information in more detail. Apart from these main classes, there are also secondary fiber targets that are based on different secondary programs in DESI (SCND targets).

The `*DESI_TARGET` columns encode the information related to which class the target belongs to. Note that some sources can be part of different classes. 

In this section, we show how to select sources based on their targeting information.

The information about different target bits is available [here](https://desidatamodel.readthedocs.io/en/latest/bitmasks.html#target-masks). By using the target bits directly, we avoid the usage of targetmasks that come from DESI software.

In [ ]:
## Function to check the bits
def check_bits_pddf(row):
    """
    Function to check the bits corresponding to the main target classes.

    Parameters
    ----------
    table : astropy table
        Table of DESI targets with required sv*desi_target columns
        
    bit : int
        Target bit from DESI global variable

    Returns
    -------
    res : numpy array
        Boolean array corresponding to the bit
    """
    # Targeting information about the DESI targeting is stored in the different desi_target columns
    sv1_desi_tgt = row['sv1_desi_target']
    sv2_desi_tgt = row['sv2_desi_target']
    sv3_desi_tgt = row['sv3_desi_target']
        
    val = (2**bit)
    res = (sv1_desi_tgt & val != 0)|(sv2_desi_tgt & val != 0)|(sv3_desi_tgt & val != 0)

    return (res)

In [ ]:
## Selecting candidates - 
## Target bits from DESI:
## 1. BGS: bit 60
## 2. LRG: bit 0
## 3. ELG: bit 1
## 4. QSO: bit 2
## 5. MWS: bit 61
## 6. Secondary Targets: bit 62

In [ ]:
targets = ['BGS', 'LRG', 'ELG', 'QSO', 'MWS', 'SCND']

In [ ]:
# MWS: Milky Way Survey (all false by constrution
bit = 61
df['MWS'] = df.apply(check_bits_pddf,axis=1)
#BGS: Bright Galaxy Survey
bit = 60
df['BGS'] = df.apply(check_bits_pddf,axis=1)
# LRG: Luminous Red Galaxies
bit = 0
df['LRG'] = df.apply(check_bits_pddf,axis=1)
# ELG: Emission Line Galaxies
bit = 1
df['ELG'] = df.apply(check_bits_pddf,axis=1)
# QSO : Quasars
bit = 2
df['QSO'] = df.apply(check_bits_pddf,axis=1)
# Secondary Targets
bit = 62
df['SCND'] = df.apply(check_bits_pddf,axis=1)

In [ ]:
df_target = df[targets]

In [ ]:
df_target_sum = df_target.sum()
df_target_sum 

In [ ]:
n_bgs = df_target_sum['BGS']
n_lrg = df_target_sum['LRG']
n_elg = df_target_sum['ELG']
n_qso = df_target_sum['QSO']

In [ ]:
import matplotlib.ticker as mtick

fig,ax = plt.subplots(1,1,figsize = (6,3))
df_target_sum.plot.bar(ax=ax,facecolor="r",xlabel="object types",ylabel="number of objects")
logfmt = mtick.LogFormatterExponent(base=10.0, labelOnlyBase=False) 
ax.yaxis.set_major_formatter(logfmt)
ax.yaxis.set_major_formatter(mtick.ScalarFormatter(useMathText=True))
ax.set_yscale('log')

In [ ]:
# Now let us look at the distribution of redshifts -

fig, axs = plt.subplots(4, 1, figsize = (8, 10))
bins = np.arange(0, 4, 0.2)

axs[0].hist(df[df['BGS']].z, color = 'C0', bins = bins, label = f'BGS: {n_bgs} sources')
axs[0].legend(fontsize = 14)
axs[0].set_ylabel("N(z)")
axs[1].hist(df[df['LRG']].z, color = 'C1', bins = bins, label = f'LRG: {n_lrg} sources')
axs[1].legend(fontsize = 14)
axs[1].set_ylabel("N(z)")
axs[2].hist(df[df['ELG']].z, color = 'C2', bins = bins, label = f'ELG: {n_elg} sources')
axs[2].legend(fontsize = 14)
axs[2].set_ylabel("N(z)")
axs[3].hist(df[df['QSO']].z, color = 'C3', bins = bins, label = f'QSO: {n_qso} sources')
axs[3].legend(fontsize = 14)
axs[3].set_ylabel("N(z)")
axs[3].set_xlabel("Redshift")

## Localisation

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,8),layout="constrained")
df[df['BGS']].plot.scatter(x="ra",y="dec",ax=ax,marker=".",color = 'C0',alpha=0.5,grid=True,label="BGS")
df[df['LRG']].plot.scatter(x="ra",y="dec",ax=ax,marker=".",color = 'C1',alpha=0.5,grid=True,label="LRG")
df[df['ELG']].plot.scatter(x="ra",y="dec",ax=ax,marker=".",color = 'C2',alpha=0.5,grid=True,label="ELG")
df[df['QSO']].plot.scatter(x="ra",y="dec",ax=ax,marker=".",color = 'C3',alpha=0.5,grid=True,label="QSO")
ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=14)
plt.gca().set_aspect('equal')

### Select LGR

In [ ]:
df = df[df['BGS']]

<a class="anchor" id="spectra_access"></a>
# Accessing and plotting the spectra of a given object

Finally, we show how to access all the available healpix-coadded spectra of a given object. We also show how to select the "best" spectrum.  
We use SPARCL, which is a fast spectral access service at Data Lab.  
Detailed ways to using SPARCL are available in this [notebook](https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/SPARCL/How_to_use_SPARCL.ipynb).

In [ ]:
# can increase or decrease this number
NSPECMIN=3

In [ ]:
selection_cut = (df['zcat_nspec'] > NSPECMIN) & (df['spectype'] == 'GALAXY')

In [ ]:
df_sel = df[selection_cut]
df_sel 

We have one spectrum per targetid

In [ ]:
df_sel_pertargetsum = df_sel.groupby(['targetid']).size()

In [ ]:
df_sel_pertargetsum.describe()

In [ ]:
#fig,ax = plt.subplots(1,1,figsize = (20,20))
#df_sel_pertargetsum.plot.barh(ax=ax,facecolor="r")

### what can be retrieved with sparcl

In [ ]:
print(client.get_default_fields())

In [ ]:
print(client.get_all_fields())

### List of target 

In [ ]:
all_targetid = df_sel.targetid.values 
all_targetid

In [ ]:
inc = ['specid', 'redshift', 'flux', 'wavelength', 'spectype', 'specprimary', 'survey', 'program', 'targetid', 'coadd_fiberstatus']

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
for targetid in all_targetid: 

    ## Retrieve Spectra
    
    row = df_sel[df_sel.targetid == targetid]
    
    _flags = ""
    for tg in targets:
        if row.iloc[0][tg]:
            _flags = _flags +  tg + ", "
           
    _ra = row.iloc[0]['ra'] 
    _dec = row.iloc[0]['dec']  
    _mag_g = row.iloc[0]['mag_g'] 
    _mag_r = row.iloc[0]['mag_r'] 
    _mag_z = row.iloc[0]['mag_z'] 
    _morphtype = row.iloc[0]['morphtype'] 
    
    textstr_ph = '\n'.join((
    r'$morph-type=%s$' % (_morphtype, ),    
    r'$ra=%.2f$ deg' % (_ra, ),
    r'$dec=%.2f$ deg' % (_dec, ),
    r'$mag\_g=%.2f$ mag' % (_mag_g, ),
    r'$mag\_r=%.2f$ mag' % (_mag_r, ),
    r'$mag\_z=%.2f$ mag' % (_mag_z, )))
    
    
    
    try:
    
        res = client.retrieve_by_specid(specid_list = [ int(targetid) ],
                                include = inc,
                                dataset_list = ['DESI-EDR'])
    
        records = res.records

        ## Select the primary spectrum
        spec_primary = np.array([records[jj].specprimary for jj in range(len(records))])
        primary_ii = np.where(spec_primary == True)[0][0]
    
    
        lam_primary = records[primary_ii].wavelength
        flam_primary = records[primary_ii].flux
    
    
        _spectype = records[primary_ii]['spectype']
        _specprimary = records[primary_ii]['specprimary']
        _targetid = records[primary_ii]['targetid']
        _redshift = records[primary_ii]['redshift']
        _specid = records[primary_ii]['specid']
        _coadd_fiberstatus = records[primary_ii]['coadd_fiberstatus']
        _program = records[primary_ii]['program']
        _survey = records[primary_ii]['survey']
        _dr = records[primary_ii]['_dr']
    
        textstr = '\n'.join((
        r'$z=%.2f$' % (_redshift, ),
        r'$flags =%s$' % (_flags, ),
        r'$program=%s$' % (_program, ),
        r'$survey=%s$' % (_survey, )))
    
        ## plot
        title = f"DESI primary target {targetid} , DR = {_dr}"
    
        fig,ax = plt.subplots(1,1,figsize = (20, 6))
        ## Plot the spectrum from each arm (B,R,Z) in blue, green, red

        ax.plot(lam_primary, flam_primary, color = 'maroon', alpha = 0.5)
        ## Over-plotting smoothed spectrum in black 
        ax.plot(lam_primary, convolve(flam_primary, Gaussian1DKernel(5)), color = 'k', lw = 2.0)
        ax.set_xlabel('$\lambda$ [$\AA$]')
        ax.set_ylabel('$F_{\lambda}$ [$10^{-17} erg\ s^{-1}\ cm^{-2}\ \AA^{-1}$]')
        # place a text box in upper left in axes coords
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=18,verticalalignment='top', bbox=props)
        ax.text(0.8, 0.95, textstr_ph, transform=ax.transAxes, fontsize=18,verticalalignment='top', bbox=props)
    
        ax.set_title(title)
        plt.show()
    
    except Exception as inst:
        print(inst)    # the exception type
    
    

<a class="anchor" id="ref"></a>
# References

* DESI EDR Documentation: https://data.desi.lbl.gov/doc/
* DESI EDR at Astro Data Lab: https://datalab.noirlab.edu/desi/
* Information about SPARCL: https://astrosparcl.datalab.noirlab.edu/sparc/
* Datamodel information: https://desidatamodel.readthedocs.io/en/latest/
    * [tiles-fuji.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/tiles-SPECPROD.html)
    * [exposures-fuji.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/exposures-SPECPROD.html)
    * [Redshift catalogs](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/index.html)
    * [zall-pix-fuji.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zall-pix-SPECPROD.html)
    * [zall-tilecumulative-fuji.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zall-tilecumulative-SPECPROD.html)
    * [zpix-SURVEY-PROGRAM.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zpix-SURVEY-PROGRAM.html)
    * [ztile-SURVEY-PROGRAM-GROUPTYPE.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/zcatalog/zpix-SURVEY-PROGRAM.html)
    * [coadd-SURVEY-PROGRAM-PIXNUM.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/coadd-SURVEY-PROGRAM-PIXNUM.html)
    * [redshift-SURVEY-PROGRAM-PIXNUM.fits](https://desidatamodel.readthedocs.io/en/latest/DESI_SPECTRO_REDUX/SPECPROD/healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/redrock-SURVEY-PROGRAM-PIXNUM.html)
* Information about DESI bitmasks: https://desidatamodel.readthedocs.io/en/latest/bitmasks.html
* Information about DESI target bitmasks: https://desidatamodel.readthedocs.io/en/latest/bitmasks.html#target-masks
* DESI Target Selection pipeline paper: [Myers et al. 2023](https://ui.adsabs.harvard.edu/abs/2023AJ....165...50M/abstract)
* Introduction to SPARCL notebook: https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/SPARCL/How_to_use_SPARCL.ipynb

---